Gerando calotas:


In [ ]:
import pandas as pd
import numpy as np
import healpy as hp
import astropy.units as u
from astropy.coordinates import SkyCoord, CartesianRepresentation, Galactic,  cartesian_to_spherical
import matplotlib.pyplot as plt
from healpy import projview


import pyccl as ccl
from scipy import integrate
import pyccl as ccl
from scipy.optimize import curve_fit
from scipy.integrate import quad

#Estamos aqui utilizando o catálogo Patheon+ (https://arxiv.org/pdf/2202.04077)


In [ ]:
# vamos definir os parâmetros de pixelização (nside=2 produz 48 calotas esféricas)

nside = 2
npix = hp.nside2npix(nside)

In [ ]:
# Vamos obter as coordenadas dos centros dos pixels para nside=2

lon, lat = hp.pix2ang(nside, range(npix), lonlat=True)

# coordenadas =  hp.pix2ang(nside, range(npix), lonlat=True)
# lp = coordenadas[0]
# bp = coordenadas[1]

galactic_coords = SkyCoord(l=lon, b=lat, frame='galactic', unit='deg')
equatorial_coords = galactic_coords.icrs #transform_to('icrs')

ra_pixel = equatorial_coords.ra.deg
dec_pixel = equatorial_coords.dec.deg

In [ ]:
#VER PIXELS em coordenadas galácticas

coord_pixel = SkyCoord(ra=ra_pixel, dec=dec_pixel, frame='icrs', unit='deg')
coordenadas_pixel = coord_pixel.galactic

l_pixel = (coordenadas_pixel.l.deg)*np.pi/180
b_pixel = (coordenadas_pixel.b.deg)*np.pi/180

In [ ]:
l_pixel[l_pixel > np.pi] = l_pixel[l_pixel > np.pi] - 2*np.pi
l_pixel[l_pixel < np.pi] = l_pixel[l_pixel < np.pi]*(-1)

In [ ]:
#conferir se está fazendo correto

figure = plt.figure(figsize=(10.0, 8.0))
ax = figure.add_subplot(111, projection='mollweide')
plt.scatter(l_pixel[20], b_pixel[20], s=8, color='blue', marker ='*')
plt.xticks(ticks=np.radians([-150, -120, -90, -60, -30, 0, 30, 60, 90, 120, 150]), labels=['150°', '120°', '90°', '60°', '30°', '0°', '330°', '300°', '270°', '240°', '210°'])
plt.grid()
plt.show()

In [ ]:
#USANDO csv
data = pd.read_csv('/home/sofia/Documentos/Pantheon/Pantheon+SH0ES.csv', sep=' ')
data


In [ ]:
Ind = []
for i in range(1701):
    Ind.append(i)

In [ ]:
data['Indice'] = Ind

In [ ]:
data

In [ ]:
ra_data = data['RA']* np.pi / 180
dec_data = data['DEC']* np.pi / 180

In [ ]:
# Raio do círculo em graus
raio = 90

n = 47
cont = 0

while cont <= n:
    
    # Coordenadas do centro do círculo/pixel em radianos
    ra_centro = ra_pixel[cont]* np.pi / 180
    dec_centro = dec_pixel[cont]* np.pi / 180
    
    # Usando a fórmula de Haversine
    delta_ra = ra_data - ra_centro
    delta_dec = dec_data - dec_centro
    
    a = np.sin(delta_dec / 2)**2 + np.cos(dec_centro) * np.cos(dec_data) * np.sin(delta_ra / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    distancia = np.degrees(c)
    objetos = data[distancia <= raio]
    
    local = f'/home/sofia/Documentos/Pantheon/Atividade 3/Hemisferios/{cont:03d}.csv'
    objetos.to_csv(local, index=False)
 
    cont += 1

In [ ]:
#Vamos visualizar cada hemisfério

hemisphere = pd.read_csv('/home/sofia/Documentos/Pantheon/Atividade 3/Hemisferios/036.csv')

rao = hemisphere['RA']
deco = hemisphere['DEC']

coord = SkyCoord(ra=rao, dec=deco, frame='icrs', unit='deg')
coordenadas = coord.galactic

lv = (coordenadas.l.deg)*np.pi/180
bv = (coordenadas.b.deg)*np.pi/180

In [ ]:
lv[lv > np.pi] = lv[lv > np.pi] - 2*np.pi
lv[lv < np.pi] = lv[lv < np.pi]*(-1)

In [ ]:
figure = plt.figure(figsize=(10.0, 8.0))
ax = figure.add_subplot(111, projection='mollweide')
plt.scatter(lv, bv, s=2, color='darkviolet', marker ='*')
plt.xticks(ticks=np.radians([-150, -120, -90, -60, -30, 0, 30, 60, 90, 120, 150]), labels=['150°', '120°', '90°', '60°', '30°', '0°', '330°', '300°', '270°', '240°', '210°'])
plt.grid()
plt.show()

Criando mapa de número de supernovas:


In [ ]:
n=47
cont=0

número = []
while cont <= n:
    local = f'/home/sofia/Documentos/Pantheon/Atividade 3/Hemisferios/{cont:03d}.csv'
    hemis = pd.read_csv(local)
    num = len(hemis)
      
    número.append(num)
    
    cont = cont + 1

In [ ]:
valores_pix = número

nside = 2
npix = hp.nside2npix(nside) 

#criar um mapa vazio e atribuir os valores de números a cada pixel dele
hp_map = np.zeros(npix)
hp_map[:len(valores_pix)] = valores_pix

hp.write_map('/home/sofia/Documentos/Pantheon/Atividade 3/Hemisferios/number_map.fits', hp_map, overwrite=True)

In [ ]:
mapa_num = hp.read_map('/home/sofia/Documentos/Pantheon/Atividade 3/Hemisferios/number_map.fits')

In [ ]:
fontsize = {
"xlabel": 15,           # Tamanho da fonte do rótulo do eixo x
"ylabel": 15,           # Tamanho da fonte do rótulo do eixo y
"title": 20,            # Tamanho da fonte do título
"xtick_label": 15,      # Tamanho da fonte dos rótulos do eixo x
"ytick_label": 15,      # Tamanho da fonte dos rótulos do eixo y
"cbar_label": 15,       # Tamanho da fonte da legenda da barra de cores
"cbar_tick_label": 15   # Tamanho da fonte dos rótulos da barra de cores
}

In [ ]:
projview(hp_map,
coord=["G"],
norm="hist",
cmap='plasma',
title='Supernovae number',
graticule=True,
graticule_labels=True,
longitude_grid_spacing=30,
projection_type="mollweide",
fontsize=fontsize)
plt.show()

Atividade 3) 

Fazendo ajuste da curva para cada hemisfério gerado (Nside=2, 48 hemisférios) e obter os parâmetros H0 e Omega_m.
Vamos começar com o modelo padrão LCDM e utilizar os demais parâmetros a partir dos dados de Plank: (https://www.aanda.org/articles/aa/pdf/2020/09/aa33910-18.pdf).


(i) Integral:

Distância de Luminosidade:
$$ m - M = 5 \cdot \log_{10}{d_L} + 25 $$

E sabemos que (comóvel)
$$ D_L(z) = \frac{c}{H0}(1+z) \int^{z}_{0} \frac{c \, dz'}{H(z')} $$

Onde 

$$H(z) = H_0 \sqrt{\Omega_m (1+z)^3 + \Omega_k (1+z)^2 + \Omega_\Lambda} $$

E em uma cosmologia $\Lambda CDM$, temos

$$\Omega_\Lambda = 1 - \Omega_m$$ 
e
 $$\Omega_k = 0$$


Então, 
$$H(z) = H_0 \sqrt{\Omega_m (1+z)^3 + 1 - \Omega_m} $$


In [ ]:
dados = pd.read_csv('/home/sofia/Documentos/Pantheon/Pantheon+SH0ES.csv', sep=' ')
Ind = []
for i in range(len(dados)):
    Ind.append(i)
    
dados['Indice'] = Ind
data = dados[(dados['zCMB'] >= 0.01)]

zCMB = np.array(data['zCMB'])
MU_SH0ES = np.array(data['MU_SH0ES'])
index = np.array(data['Indice'])

cove = np.genfromtxt('/home/sofia/Documentos/Pantheon/Pantheon+SH0ES_STAT+SYS.cov', skip_header=1)
cov = np.reshape(cove, (1701, 1701))

In [ ]:
def mu_fit(z, H0, Om):
    c = 299793.458
    Om_lamb = 1 - Om

    def integrando(z):
        return 1 / np.sqrt(Om * (1 + z)**3 + Om_lamb)

    mu_values = []

    for z_val in z:
        I, _ = integrate.quad(integrando, 0, z_val) 
        dL = (c / H0) * (1 + z_val) * I
        mu = 5 * np.log10(dL) + 25
        mu_values.append(mu)

    return np.array(mu_values)

In [ ]:
I = np.int64(index)
E = cov[np.ix_(I, I)]

p0 = [70, 0.3]

curve, cove = curve_fit(mu_fit, zCMB, MU_SH0ES, p0=p0, sigma=E, method='trf')

In [ ]:
curve

In [ ]:
index = []
zCMB = []
MU_SHOES = []

for i in range(48):
    di = f'/home/sofia/Documentos/Pantheon/Atividade 3/Hemisferios/{i:03d}.csv'

    hemis = pd.read_csv(di)
    index.append(hemis['Indice'].values)
    zCMB.append(hemis['zCMB'].values)
    MU_SHOES.append(hemis['MU_SH0ES'].values)

In [ ]:
cove = np.genfromtxt('/home/sofia/Documentos/Pantheon/Pantheon+SH0ES_STAT+SYS.cov', skip_header=1)
cov = np.reshape(cove, (1701, 1701))

In [ ]:
H0 = []
Om = []

for i in range(48):
    I_h = np.int64(index[i])
    E_h = cov[np.ix_(I_h, I_h)]
    
    p0 = [70, 0.3]

    curve, cove = curve_fit(mu_fit, zCMB[i], MU_SHOES[i], p0=p0, sigma=E_h, method='trf')
    print(curve)
    H0.append(curve[1])
    Om.append(curve[0])

In [ ]:
H0_pix = H0
Om_pix = Om

nside = 2  
npix = hp.nside2npix(nside) 

#criar um mapa vazio e atribuir os valores de números a cada pixel dele
H0_map = np.zeros(npix)
H0_map[:len(H0_pix)] = H0_pix

Om_map = np.zeros(npix)
Om_map[:len(Om_pix)] = Om_pix

hp.write_map('/home/sofia/Documentos/Pantheon/Atividade 3/H0_map_integral.fits', H0_map, overwrite=True)
hp.write_map('/home/sofia/Documentos/Pantheon/Atividade 3/Om_map_integral.fits', Om_map, overwrite=True)

In [ ]:
fontsize = {
"xlabel": 15,           # Tamanho da fonte do rótulo do eixo x
"ylabel": 15,           # Tamanho da fonte do rótulo do eixo y
"title": 20,            # Tamanho da fonte do título
"xtick_label": 15,      # Tamanho da fonte dos rótulos do eixo x
"ytick_label": 15,      # Tamanho da fonte dos rótulos do eixo y
"cbar_label": 15,       # Tamanho da fonte da legenda da barra de cores
"cbar_tick_label": 15   # Tamanho da fonte dos rótulos da barra de cores
}

In [ ]:
projview(H0_map,
coord=["G"],
norm="hist",
cmap='jet',
graticule=True,
graticule_labels=True,
longitude_grid_spacing=30,
projection_type="mollweide",
fontsize=fontsize)

projview(Om_map,
coord=["G"],
norm="hist",
cmap='jet',
graticule=True,
graticule_labels=True,
longitude_grid_spacing=30,
projection_type="mollweide",
fontsize=fontsize)

plt.show()

(ii)Pyccl: 

(vide https://ccl.readthedocs.io/en/latest/api/pyccl.cosmology.html#pyccl.cosmology.Cosmology). 

In [ ]:
dados = pd.read_csv('/home/sofia/Documentos/Pantheon/Pantheon+SH0ES.csv', sep=' ')

Ind = []
for i in range(len(dados)):
    Ind.append(i)
    
dados['Indice'] = Ind

data = dados[(dados['zCMB'] >= 0.01)]

zCMB = np.array(data['zCMB'])
MU_SHOES = np.array(data['MU_SH0ES'])
index = np.array(data['Indice'])

cove = np.genfromtxt('/home/sofia/Documentos/Pantheon/Pantheon+SH0ES_STAT+SYS.cov', skip_header=1)
cov = np.reshape(cove, (1701, 1701))

In [ ]:
def mu_fit(z, Omc, h):
    
    # Omega_C é a densidade de matéria escura
    # Omega_b é a densidade de matéria bariônica
    # h é o parâmetro de Hubble reduzido
    # sigma8 é a amplitude das flutuações de densidade em escalas de 8Mpc
    # n_s é o índice de espectro das perturbações primordiais
    # se tratram de valores típicos de um modelo cosmológico padrão (LCDM - Lambda Cold Dark Matter)
    cosmo = ccl.Cosmology(
        Omega_c=Omc, Omega_b=0.0494, h=h, sigma8=0.8120, n_s=0.9649)
    
    #a é o fator de escala
    a = 1. / (1. + z)
    
    return ccl.distance_modulus(cosmo, a)

In [ ]:
#convertendo index para o tipo int64 
I = np.int64(index)
#selecionando a submatriz de covariância correspondente
E = cov[np.ix_(I, I)]
curve, cove = curve_fit(mu_fit, zCMB, MU_SHOES, sigma=E, method='trf')

In [ ]:
curve

In [ ]:
final = curve[0],curve[1]*100

In [ ]:
final

(iii) Fazendo o best fit dos 48 hemisférios:

In [ ]:
dados = pd.read_csv('/home/sofia/Documentos/Pantheon/Pantheon+SH0ES.csv', sep=' ')
Ind = []
for i in range(len(dados)):
    Ind.append(i)
    
dados['Indice'] = Ind

data = dados[(dados['zCMB'] >= 0.01)]

In [ ]:
data

In [ ]:
index = []
zCMB = []
MU_SHOES = []

for i in range(48):
    di = f'/home/sofia/Documentos/Pantheon/Atividade 3/Hemisferios/{i:03d}.csv'
    hemis = pd.read_csv(di)
    index.append(hemis['Indice'].values)
    zCMB.append(hemis['zCMB'].values)
    MU_SHOES.append(hemis['MU_SH0ES'].values)

In [ ]:
index

In [ ]:
cove = np.genfromtxt('/home/sofia/Documentos/Pantheon/Pantheon+SH0ES_STAT+SYS.cov', skip_header=1)
cov = np.reshape(cove, (1701, 1701))

In [ ]:
def mu_fit(z, Omc, h):
    
    cosmo = ccl.Cosmology(
        Omega_c=Omc, Omega_b=0.0494, h=h, sigma8=0.8120, n_s=0.9649)
    
    a = 1. / (1. + z)
    
    return ccl.distance_modulus(cosmo, a)

h0 = []
Om = []

In [ ]:
for i in range(48):
    I_h = np.int64(index[i])
    E_h = cov[np.ix_(I_h, I_h)]
    curve, cove = curve_fit(mu_fit, zCMB[i], MU_SHOES[i], sigma=E_h, method='trf')
    h0.append(curve[1])
    Om.append(curve[0])

In [ ]:
H0 = 100 * np.array(h0)
#H0=h*100 Km/s/Mpc
Om_tot = np.array(Om)+0.0494

In [ ]:
print(H0)

In [ ]:
H0_pix = H0
Om_pix = Om_tot

nside = 2  
npix = hp.nside2npix(nside) 

#criar um mapa vazio e atribuir os valores de números a cada pixel dele
H0_map = np.zeros(npix)
H0_map[:len(H0_pix)] = H0_pix

Om_map = np.zeros(npix)
Om_map[:len(Om_pix)] = Om_pix

hp.write_map('/home/sofia/Documentos/Pantheon/Atividade 3/H0_map_pyccl.fits', H0_map)
hp.write_map('/home/sofia/Documentos/Pantheon/Atividade 3/Om_map_pyccl.fits', Om_map)

In [ ]:
mapa_H0 = hp.read_map('/home/sofia/Documentos/Pantheon/Atividade 3/H0_map_pyccl.fits')
mapa_Om = hp.read_map('/home/sofia/Documentos/Pantheon/Atividade 3/Om_map_pyccl.fits')

In [ ]:
fontsize = {
"xlabel": 15,           # Tamanho da fonte do rótulo do eixo x
"ylabel": 15,           # Tamanho da fonte do rótulo do eixo y
"title": 20,            # Tamanho da fonte do título
"xtick_label": 15,      # Tamanho da fonte dos rótulos do eixo x
"ytick_label": 15,      # Tamanho da fonte dos rótulos do eixo y
"cbar_label": 15,       # Tamanho da fonte da legenda da barra de cores
"cbar_tick_label": 15   # Tamanho da fonte dos rótulos da barra de cores
}

In [ ]:
projview(mapa_H0,
coord=["G"],
norm="hist",
cmap='jet',
graticule=True,
graticule_labels=True,
longitude_grid_spacing=30,
projection_type="mollweide",
fontsize=fontsize)

projview(mapa_Om,
coord=["G"],
norm="hist",
cmap='jet',
graticule=True,
graticule_labels=True,
longitude_grid_spacing=30,
projection_type="mollweide",
fontsize=fontsize)

plt.show()